# VarnaAir - Open Programme

## Introduction ℹ️


## Libraries 📚
First, let's begin by importing the libraries and checking their versions.

In [106]:
import pandas
import requests

## Data provisioning 📦

In [107]:
url = "https://api.openaq.org/v3/locations"

headers = {
    "X-API-Key": "c1061fd64a9bd88562b4bb823b7900ccdfd263a9a7aa54614de834dac611a4ca"
}
params = {
    "coordinates": "43.224389,27.915733",  # center of Varna
    "radius": 5000,                        # 5 km radius
    "limit": 100,                         
}

response = requests.get(url, headers=headers, params=params)

data = response.json().get("results", [])
df_nearby_stations = pandas.DataFrame(data)
print(df_nearby_stations[["id", "name", "locality"]])

        id                         name              locality
0     8843  AMS SOU Angel Kanchev-Varna                 Варна
1  2162113           AMS Chaika - Varna  National air network


In [137]:
url = "https://api.openaq.org/v3/locations/8843/latest"
        
response = requests.get(url, headers=headers)
response.raise_for_status()
    
data = response.json()
results = data.get("results", [])
    
if results:
    df_latest = pandas.DataFrame(results)
        
    df_latest = df_latest.assign(
        sensor_id=df_latest['sensorsId'],
        utc_time=df_latest['datetime'].apply(lambda x: x['utc']),
        local_time=df_latest['datetime'].apply(lambda x: x['local']),
        latitude=df_latest['coordinates'].apply(lambda x: x['latitude']),
        longitude=df_latest['coordinates'].apply(lambda x: x['longitude'])
    ).drop(columns=['datetime', 'coordinates'])
else:
    df_latest = pandas.DataFrame()

df_latest

,value,sensorsId,locationsId,sensor_id,utc_time,local_time,latitude,longitude
0,73.45,25777,8843,25777,2025-06-15T18:00:00Z,2025-06-15T21:00:00+03:00,43.224389,27.915733
1,10.31,25778,8843,25778,2025-06-15T18:00:00Z,2025-06-15T21:00:00+03:00,43.224389,27.915733
2,6.79,25776,8843,25776,2025-06-15T18:00:00Z,2025-06-15T21:00:00+03:00,43.224389,27.915733
3,370.00,25779,8843,25779,2025-06-15T18:00:00Z,2025-06-15T21:00:00+03:00,43.224389,27.915733
4,11.18,25774,8843,25774,2025-06-15T18:00:00Z,2025-06-15T21:00:00+03:00,43.224389,27.915733
5,7.02,25775,8843,25775,2025-06-15T18:00:00Z,2025-06-15T21:00:00+03:00,43.224389,27.915733
6,2.47,4272879,8843,4272879,2024-03-11T11:00:00Z,2024-03-11T13:00:00+02:00,43.224389,27.915733


In [146]:
base_url = "https://api.openaq.org/v3/sensors/{sensor_id}/measurements"

# sensor ids for Angel Kanchev
sensor_ids = [25775, 25778, 25776, 25779, 25774, 25777, 4272879]  

sensor_data_frames = []

for sensor_id in sensor_ids:
    url = base_url.format(sensor_id=sensor_id)
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    data = response.json()
    results = data.get("results", [])

    if results:
        df = pandas.DataFrame(results)

        df = df.assign(
            sensor_id=sensor_id,
            datetime_from_utc=df['period'].apply(lambda x: x['datetimeFrom']['utc'] if x else None),
            datetime_from_local=df['period'].apply(lambda x: x['datetimeFrom']['local'] if x else None),
            datetime_to_utc=df['period'].apply(lambda x: x['datetimeTo']['utc'] if x else None),
            datetime_to_local=df['period'].apply(lambda x: x['datetimeTo']['local'] if x else None),
            parameter_name=df['parameter'].apply(lambda x: x.get('name') if isinstance(x, dict) else None),
            parameter_units=df['parameter'].apply(lambda x: x.get('units') if isinstance(x, dict) else None)
        ).drop(columns=['flagInfo', 'parameter', 'period', 'summary', 'coverage', 'coordinates'])

        sensor_data_frames.append(df)

combined_df = pandas.concat(sensor_data_frames, ignore_index=True)
combined_df

,value,sensor_id,datetime_from_utc,datetime_from_local,datetime_to_utc,datetime_to_local,parameter_name,parameter_units
0,18.38,25775,2020-04-20T17:00:00Z,2020-04-20T20:00:00+03:00,2020-04-20T18:00:00Z,2020-04-20T21:00:00+03:00,pm25,µg/m³
1,19.34,25775,2020-04-20T18:00:00Z,2020-04-20T21:00:00+03:00,2020-04-20T19:00:00Z,2020-04-20T22:00:00+03:00,pm25,µg/m³
2,18.33,25775,2020-04-20T19:00:00Z,2020-04-20T22:00:00+03:00,2020-04-20T20:00:00Z,2020-04-20T23:00:00+03:00,pm25,µg/m³
3,17.23,25775,2020-04-20T20:00:00Z,2020-04-20T23:00:00+03:00,2020-04-20T21:00:00Z,2020-04-21T00:00:00+03:00,pm25,µg/m³
4,19.66,25775,2020-04-20T21:00:00Z,2020-04-21T00:00:00+03:00,2020-04-20T22:00:00Z,2020-04-21T01:00:00+03:00,pm25,µg/m³
5,19.21,25775,2020-04-20T22:00:00Z,2020-04-21T01:00:00+03:00,2020-04-20T23:00:00Z,2020-04-21T02:00:00+03:00,pm25,µg/m³
6,17.00,25775,2020-04-20T23:00:00Z,2020-04-21T02:00:00+03:00,2020-04-21T00:00:00Z,2020-04-21T03:00:00+03:00,pm25,µg/m³
7,19.03,25775,2020-04-21T00:00:00Z,2020-04-21T03:00:00+03:00,2020-04-21T01:00:00Z,2020-04-21T04:00:00+03:00,pm25,µg/m³
8,20.46,25775,2020-04-21T01:00:00Z,2020-04-21T04:00:00+03:00,2020-04-21T02:00:00Z,2020-04-21T05:00:00+03:00,pm25,µg/m³
9,21.15,25775,2020-04-21T02:00:00Z,2020-04-21T05:00:00+03:00,2020-04-21T03:00:00Z,2020-04-21T06:00:00+03:00,pm25,µg/m³
